# BINARY FORMATING

In [12]:
import pandas as pd

In [13]:
df = pd.read_excel(r'C:\Users\sumeet4.singh\Desktop\senti_project\1. Feedback_Project\Data\TRAIN(CAT).xlsx')

In [14]:
df.sample

<bound method NDFrame.sample of                                      feedback_comments  \
0    doctors should be here on time..  I had to wai...   
1    long waiting for dietitian and doctors consult...   
2    consulting not done in time so couldn't  compl...   
3    there was carelessness while attending the pat...   
4                   Excellent team at Executive health   
..                                                 ...   
245  Discharge instructions were clear and well exp...   
246    Admission staff were professional and efficient   
247    The discharge process took longer than expected   
248  I was well informed about the steps during my ...   
249  The staff was helpful during admission, and th...   

                                    Categories  
0                            Doctor Experience  
1    Interaction with Staff, Doctor Experience  
2                              Service Quality  
3                              Service Quality  
4                       In

In [15]:
df.columns

Index(['feedback_comments', 'Categories'], dtype='object')

In [16]:
import pandas as pd

# Load the DataFrame from the provided Excel file
file_path = r'C:\Users\sumeet4.singh\Desktop\senti_project\1. Feedback_Project\Data\TRAIN(CAT).xlsx'
df = pd.read_excel(file_path)

# List of candidate labels (categories you want to create binary columns for)
candidate_labels = [
    "Service Quality", "Doctor Experience", "Nursing Experience",
    "Interaction with Staff", "Lab & Radiology Services", "Facilities and Infrastructure",
    "Billing and Payments", "Appointment Process", "Admission & Discharge",
    "Food & Beverages", "Housekeeping/PCA/PTA", "Others"
]

# Function to create binary columns based on the categories column
def create_binary_columns(df, candidate_labels):
    for label in candidate_labels:
        # Create a new column for each category
        df[label] = df['Categories'].apply(lambda x: 1 if label in str(x) else 0)
    return df

# Apply the function to add binary columns to the dataframe
df = create_binary_columns(df, candidate_labels)

# Save the updated DataFrame with the binary columns to a new Excel file
output_path = r'C:\Users\sumeet4.singh\Desktop\senti_project\1. Feedback_Project\Data\TRAIN(BINARY).xlsx'
df.to_excel(output_path, index=False)

# Display the resulting DataFrame to check
print(df.head())  # Display the first few rows of the updated dataframe

                                   feedback_comments  \
0  doctors should be here on time..  I had to wai...   
1  long waiting for dietitian and doctors consult...   
2  consulting not done in time so couldn't  compl...   
3  there was carelessness while attending the pat...   
4                 Excellent team at Executive health   

                                  Categories  Service Quality  \
0                          Doctor Experience                0   
1  Interaction with Staff, Doctor Experience                0   
2                            Service Quality                1   
3                            Service Quality                1   
4                     Interaction with Staff                0   

   Doctor Experience  Nursing Experience  Interaction with Staff  \
0                  1                   0                       0   
1                  1                   0                       1   
2                  0                   0                       0   


# NOW FINE TUNE FOR CATEGORIZATION

In [17]:
import torch
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from tqdm import tqdm

In [18]:
# Load the DataFrame from the provided Excel file
df = pd.read_excel(r'C:\Users\sumeet4.singh\Desktop\senti_project\1. Feedback_Project\Data\TRAIN(BINARY).xlsx')

In [19]:
# Sample candidate labels for categorization (as per your list)
candidate_labels = [
    "Service Quality", "Doctor Experience", "Nursing Experience",
    "Interaction with Staff", "Lab & Radiology Services", "Facilities and Infrastructure",
    "Billing and Payments", "Appointment Process", "Admission & Discharge",
    "Food & Beverages", "Housekeeping/PCA/PTA", "Others"
]

In [20]:
# Ensure the 'feedback_comments' column is in string format
df['feedback_comments'] = df['feedback_comments'].astype(str)

# Replace 'nan' (as a string) with an empty string (or another placeholder like 'No comment')
df['feedback_comments'] = df['feedback_comments'].replace('nan', '').replace('No comment', '').replace('  ', ' ')

# Function to clean and prepare the comment text
def clean_comment(comment):
    # Remove leading/trailing spaces and newlines
    comment = comment.strip()
    # Normalize multiple spaces to a single space
    comment = re.sub(r'\s+', ' ', comment)
    # Remove any non-ASCII characters (if needed)
    comment = ''.join([c for c in comment if ord(c) < 128])
    return comment

# Apply cleaning function to all comments
df['feedback_comments'] = df['feedback_comments'].apply(clean_comment)

In [21]:
# Extract the binary labels (12 columns for each category)
labels_columns = candidate_labels  # These columns contain binary labels

# Directly use the binary labels from the DataFrame
df_labels = df[labels_columns].values  # This is already in binary format for each category

# Function to preprocess text using the BART tokenizer
tokenizer = AutoTokenizer.from_pretrained(r"C:\Users\sumeet4.singh\Desktop\senti_project\1. Feedback_Project\Scripts\pre_trained_BART")

def preprocess_data(texts, tokenizer, max_length=128):
    return tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['feedback_comments'].tolist(), df_labels.tolist(), test_size=0.2)

train_inputs = preprocess_data(train_texts, tokenizer)
val_inputs = preprocess_data(val_texts, tokenizer)

In [22]:
# Convert to torch datasets
class FeedbackDataset(Dataset):
    def __init__(self, inputs, labels):
        self.input_ids = inputs['input_ids']
        self.attention_mask = inputs['attention_mask']
        # Convert labels to Float type here (for multi-label classification)
        self.labels = torch.tensor(labels, dtype=torch.float32)
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }
    
    def __len__(self):
        return len(self.labels)

train_dataset = FeedbackDataset(train_inputs, train_labels)
val_dataset = FeedbackDataset(val_inputs, val_labels)

In [23]:
# Load pre-trained BART model for sequence classification (multi-label classification)
model = AutoModelForSequenceClassification.from_pretrained(
    r'C:\Users\sumeet4.singh\Desktop\senti_project\1. Feedback_Project\Scripts\pre_trained_BART',
    num_labels=len(candidate_labels),
    problem_type="multi_label_classification"  # Explicitly specify multi-label classification
)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at C:\Users\sumeet4.singh\Desktop\senti_project\1. Feedback_Project\Scripts\pre_trained_BART and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# Set up the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  # Adjust epochs based on your needs
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    no_cuda=True  # Explicitly disable CUDA (GPU) usage
)

C:\Users\sumeet4.singh\AppData\Local\miniconda3\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\sumeet4.singh\AppData\Local\miniconda3\Lib\site-packages\transformers\training_args.py:1583: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [27]:
# Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Fine-tune the model
trainer.train()

C:\Users\sumeet4.singh\AppData\Local\Temp\ipykernel_4072\107209988.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.288300,0.342350
2,0.265900,0.338741
3,0.248300,0.328245


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight'].


TrainOutput(global_step=75, training_loss=0.27484471003214517, metrics={'train_runtime': 530.3433, 'train_samples_per_second': 1.131, 'train_steps_per_second': 0.141, 'total_flos': 163022168678400.0, 'train_loss': 0.27484471003214517, 'epoch': 3.0})

In [28]:
# Directory where the model and tokenizer will be saved
save_dir = r'C:\Users\sumeet4.singh\Desktop\senti_project\1. Feedback_Project\Scripts\fine_tuned_bart_model'

# Save the fine-tuned model
trainer.save_model(save_dir)

# Save the tokenizer to the same directory
tokenizer.save_pretrained(save_dir)

print("Fine-tuning complete. Model and tokenizer saved!")

Fine-tuning complete. Model and tokenizer saved!


# USE THE FINE TUNE MODEL NOW ON (BENCHMARKING DATA)

In [31]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm

# Load the fine-tuned PyTorch model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(r'C:\Users\sumeet4.singh\Desktop\senti_project\1. Feedback_Project\Scripts\fine_tuned_bart_model')
tokenizer = AutoTokenizer.from_pretrained(r'C:\Users\sumeet4.singh\Desktop\senti_project\1. Feedback_Project\Scripts\fine_tuned_bart_model')

# Move model to device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load your Excel file
df = pd.read_excel(r'C:\Users\sumeet4.singh\Desktop\senti_project\1. Feedback_Project\Data\Benchmarking data.xlsx')

# Sample candidate labels for categorization
candidate_labels = [
    "Service Quality", "Doctor Experience", "Nursing Experience",
    "Interaction with Staff", "Lab & Radiology Services", "Facilities and Infrastructure",
    "Billing and Payments", "Appointment Process", "Admission & Discharge",
    "Food & Beverages", "Housekeeping/PCA/PTA", "Others"
]

# Function to preprocess the text (same as the one used during training)
def preprocess_text(text, tokenizer, max_length=128):
    return tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors='pt')

# Function to make predictions for a single comment (PyTorch version)
def predict_category_pt(text, model, tokenizer, candidate_labels, threshold=0.5):
    # Preprocess the input text
    inputs = preprocess_text(text, tokenizer)
    
    # Move the input tensors to the same device as the model (GPU or CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Ensure that the model is in evaluation mode
    model.eval()
    
    # Make predictions (output is logits)
    with torch.no_grad():  # Disable gradient calculations for inference
        outputs = model(**inputs)
    
    # Get predicted logits and apply a sigmoid to get probabilities
    logits = outputs.logits
    probabilities = torch.sigmoid(logits)  # Apply sigmoid for multi-label classification
    
    # Convert probabilities to labels (1 for categories with probability > threshold, else 0)
    predicted_labels = (probabilities > threshold).int().squeeze().tolist()
    
    # Map the predictions to the corresponding categories
    predicted_categories = [candidate_labels[i] for i in range(len(candidate_labels)) if predicted_labels[i] == 1]
    
    return predicted_categories, probabilities.squeeze().tolist()

# Function to categorize all feedback comments in the DataFrame
def categorize_feedback_pt(df, model, tokenizer, candidate_labels, threshold=0.2):
    all_predictions = []
    all_probabilities = {label: [] for label in candidate_labels}
    
    for comment in tqdm(df['feedback'], desc="Categorizing Feedback"):  # Assuming column name is 'feedback_comments'
        predicted_categories, probabilities = predict_category_pt(comment, model, tokenizer, candidate_labels, threshold)
        all_predictions.append(predicted_categories)
        
        for i, label in enumerate(candidate_labels):
            all_probabilities[label].append(probabilities[i])
    
    df['Categories'] = all_predictions
    for label in candidate_labels:
        df[f'{label}_score'] = all_probabilities[label]
    
    return df

# Ensure 'feedback_comments' column exists in the DataFrame
if 'feedback' in df.columns:
    df = categorize_feedback_pt(df, model, tokenizer, candidate_labels)
else:
    print("Error: The DataFrame does not contain a 'feedback' column")

# Save the updated DataFrame to a new Excel file
df.to_excel(r'C:\Users\sumeet4.singh\Desktop\senti_project\1. Feedback_Project\Data\Benchmarking data(CAT).xlsx', index=False)

print("Categorization complete with scores. Results saved.")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1', '2': 'LABEL_2', '3': 'LABEL_3', '4': 'LABEL_4', '5': 'LABEL_5', '6': 'LABEL_6', '7': 'LABEL_7', '8': 'LABEL_8', '9': 'LABEL_9', '10': 'LABEL_10', '11': 'LABEL_11'}. The number of labels wil be overwritten to 12.
Categorizing Feedback: 100%|█████████████████████████████████████████████████████████| 345/345 [00:37<00:00,  9.23it/s]


Categorization complete with scores. Results saved.
